In [1]:
from datasets import load_dataset
import json
from sentence_transformers import SentenceTransformer
from datasets import load_dataset

SBERT_MODEL = "all-MiniLM-L6-v2"
import pandas as pd
import numpy as np
import scipy.spatial as sp
from collections import defaultdict
from tqdm import tqdm
import faiss
from collections import ChainMap


In [30]:

THRESHOLD = 0.72
B_SIZE = 500


In [80]:
np.argwhere?

In [94]:
def load_vectorizer(model=SBERT_MODEL):
    return SentenceTransformer(model)


def vectorize_text(model, texts):
    return model.encode(texts, show_progress_bar=True)

def build_index(vector):
    
    d = vector.shape[1]
    index = faiss.IndexFlatIP(d)
    faiss.normalize_L2(rot_vector)
    index.add(rot_vector)
    return index
    

def get_conversations(rot_vector, posts_vector):
    
    index = build_index(rot_vector)
    num_result=5
    faiss.normalize_L2(posts_vector)
    matching_dict = defaultdict(list)
    for idx in tqdm(range(0, posts_vector.shape[0], B_SIZE)):
        I, D = index.search(posts_vector[idx:idx+B_SIZE],num_result)
        sim_indices = np.argwhere(I >= THRESHOLD)
        #return sim_indices
        for k, j in sim_indices:
            matching_dict[k+idx].append({D[k][j]:I[k][j]})

    return matching_dict

def get_top_n(data,n=2):
    
    data = dict(ChainMap(*data))
    data = {k: v for k, v in sorted(data.items(), key=lambda data: data[1],reverse=True)}
    return list(data.keys())[:n]
    

def convert_to_json(match_dict,all_rots,reddit_df):
    all_data = []
    posts = reddit_df['title'].values
    permalinks = reddit_df['permalink'].values
    for post_id,rot_list in match_dict.items():
        rot_ids = get_top_n(rot_list)
        ps_rots = [all_rots[idx] for idx in rot_ids]
        all_data.append({"user":posts[post_id], "ROTs":ps_rots,"permalink":permalinks[post_id]})
    
    return all_data

- Now I am matching K rots with all posts and repeating this process
    - This will cause similar rots to match same posts again 
    - Inorder to remove this, Take all rots compare with batches of posts
    - implement FAISS if necessary

In [4]:
dataset = load_dataset("allenai/prosocial-dialog",split="train")
conf_df = pd.read_csv("/home/shahul/Data/one-million-reddit-confessions.csv")

Using custom data configuration allenai--prosocial-dialog-ebbad39ca08b6d44
Found cached dataset json (/home/shahul/.cache/huggingface/datasets/allenai___json/allenai--prosocial-dialog-ebbad39ca08b6d44/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)
/tmp/ipykernel_108573/1411690863.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  conf_df = pd.read_csv("/home/shahul/Data/one-million-reddit-confessions.csv")


In [5]:
conf_df_filtered = conf_df.query("score>3.0")
conf_df_filtered["title_len"] = conf_df_filtered['title'].map(lambda x : len(x.split()))
conf_df_filtered = conf_df_filtered.query("title_len>5")

/tmp/ipykernel_108573/162276847.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  conf_df_filtered["title_len"] = conf_df_filtered['title'].map(lambda x : len(x.split()))


In [6]:
rots = [item["rots"] for item in dataset]
rots = set([x for item in rots for x in item])

In [7]:
rots = list(rots)

In [8]:
model = load_vectorizer()

In [9]:
rot_vector = vectorize_text(model, rots)

Batches:   0%|          | 0/3630 [00:00<?, ?it/s]

In [13]:
posts = conf_df_filtered['title'].values.tolist()
posts_vector = vectorize_text(model,posts)

Batches:   0%|          | 0/5431 [00:00<?, ?it/s]

In [57]:
conf_df_filtered.id.nunique()

173770

In [91]:
match_dict = get_conversations(rot_vector,posts_vector[:1000])

100%|█████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.67s/it]


In [95]:
json_data = convert_to_json(match_dict,rots,conf_df_filtered.head(1000))

In [96]:
json_data

[{'user': 'i accused a family member of something they didn’t do',
  'ROTs': ['It is understandable to be accused of something you did not do.',
   "You should get a lawyer if you're accused of something you didn't do."],
  'permalink': 'https://old.reddit.com/r/confession/comments/pwv3h1/i_accused_a_family_member_of_something_they_didnt/'},
 {'user': 'I told my friend who is abused to get hit by their Dad',
  'ROTs': ["You should tell somebody if you've been abused."],
  'permalink': 'https://old.reddit.com/r/confession/comments/ptpc6d/i_told_my_friend_who_is_abused_to_get_hit_by/'},
 {'user': 'I stole funds from my school a couple of years ago',
  'ROTs': ["It's wrong to steal money from your school."],
  'permalink': 'https://old.reddit.com/r/confession/comments/pse1oe/i_stole_funds_from_my_school_a_couple_of_years_ago/'},
 {'user': 'I’m guilty of hating it when my friends are with their partners.',
  'ROTs': ["It's ok to dislike your partner's friends"],
  'permalink': 'https://old